# Assignment: Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?

  >The paper is about the process of data cleaning, specifically about tidy datasets, where each variable is a column, each observation is a row, and each unit group is a table.

  2. Read the introduction. What is the "tidy data standard" intended to accomplish?

  >The tidy data standard intends to make data analysis easier by providing a standardized way of organizing values within a dataset.

  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."

  >The first sentence means that tidy data sets are all similar in one way, but messy data sets are all uniquely different from one another.

  >The second sentence means that defining what the rows and columns are for datasets is easy, but defining the observational units is not as obvious.

  4. Read Section 2.2. How does Wickham define values, variables, and observations?

  > Values are numeric or categorical/strings. Variables are a collection of values that measure the same thing. An obersvation is a collection of values that measures the observational unit.

  5. How is "Tidy Data" defined in section 2.3?

  > Variables are columns, obervations are rows, and observational units are tables.

  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?

  >1. column headers are values, not variables.
  >2. multiple variables are stored in one column.
  >3. variables are stored in both rows and columns.
  >4. multiple observational units are stored in one table.
  >5. a single observational unit is stored in multiple tables.

  > In table 4, the columns all define income and frequency, which should be separated into individual rows.
  > Melting a dataset is the process of converting column-value variables into rows.

  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

  > Table 11 is messy because there are days as columns, but they should be values instead.
  > Table 12 is molten because each tmax and tmin measurement lies on the same day, so it does not need a separate observational row. It is also tidy because all the entries contain unique attributes and has no variable names as columns.

  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

  > The chicken-and-egg problem is that people will only build tools with tidy data, and so those tools will only be useful with tidy data. Wickham hopes that there will be new frameworks and tools that expand off tidy data rather than only relying on it as the sole source of data representation.

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)

    >0 missing values.

2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.


    >I combined all the incidents that took place on a watercraft or boat to be just under the boat variable in order to consolidate the different types. I then changed all the missing or unknown data to be null values to clean the data.


3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

    >I set the sentence length to 0 when the sentence type was 4 because those cases had charges that were pending, dismissed, or deferred. I set the sentence length to null values when the sentence type was 9 becuase those were not applicable.

In [20]:
#cleaning for q2.1
df = pd.read_csv('./data/airbnb_hw.csv', low_memory = False)
price = df['Price']
#price.value_counts()
price = price.str.replace(',', '')
price = pd.to_numeric(price, errors = 'coerce')

print('missing: ', sum(price.isnull())) 

missing:  0


In [23]:
#cleaning for q2.2
df = pd.read_csv('./data/sharks.csv', low_memory = False)
type = df['Type']
#type.value_counts()
type = type.replace(['Sea Disaster', 'Watercraft', 'Boating', 'Boatomg'], 'Boat') # All boats
type = type.replace(['Invalid', 'Questionable', 'Unconfirmed', 'Unverified', 'Under investigation'], np.nan) # All null

type.value_counts()

Type
Unprovoked    4716
Provoked       593
Boat           583
Name: count, dtype: int64

In [25]:
#cleaning for q2.3
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df = pd.read_csv(url,low_memory=False) 

In [28]:
release = df['WhetherDefendantWasReleasedPretrial']
# release.value_counts()
release = release.replace(9, np.nan)

# sum(release.isnull()) 
release.value_counts()

WhetherDefendantWasReleasedPretrial
1.0    19154
0.0     3801
Name: count, dtype: int64

In [35]:
#cleaning for q2.4
length = df['ImposedSentenceAllChargeInContactEvent']
type = df['SentenceTypeAllChargesAtConvictionInContactEvent']

# print(length.value_counts())
# print(type.value_counts())

length = pd.to_numeric(length, errors = 'coerce')
print(sum(length.isnull()))

length = length.mask(type == 4, 0) # pending, dismissed, or deferred charges
length = length.mask(type == 9, np.nan) # not applicable
print(sum(length.isnull()))

9053
274


**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?